# Carga de datos

In [ ]:
import pandas as pd


clients = pd.read_csv("datos/caso_2/clients.csv")
depots = pd.read_csv("datos/caso_2/depots.csv")
depots = depots.drop(columns=["DepotID"])
vehicles = pd.read_csv("datos/caso_2/vehicles.csv")


print(clients.head())
print(depots.head())
print(vehicles.head())

# Funciones auxiliares

In [ ]:
import openrouteservice
from openrouteservice import convert
from haversine import haversine


def dehhmm_a_minutos(hhmm):
    """
    Convierte un tiempo en formato de horas y minutos (HHMM) a minutos.
    :param dehhmm: Tiempo en formato HHMM.
    :return: Tiempo en minutos.
    """
    horas = hhmm // 100
    minutos = hhmm % 100
    return horas * 60 + minutos


# recibe "hh:mm-hh:mm" y devuelve una lista de tuplas con los tiempos en minutos
def convertir_hora_a_minutos(rango_horas):
    """
    Convierte un rango de horas en formato "hh:mm-hh:mm" a una lista de tuplas con los tiempos en minutos.
    :param rango_horas: Rango de horas en formato "hh:mm-hh:mm".
    :return: tiempos en minutos.
    """
    inicio, fin = rango_horas.split('-')
    inicio_min = dehhmm_a_minutos(int(inicio.replace(':', '')))
    fin_min = dehhmm_a_minutos(int(fin.replace(':', '')))
    return [inicio_min, fin_min]

def get_distancia_y_tiempo(api_key, start_coords, end_coords):
    # Reemplaza con tu clave de API
    api_key = api_key

    client = openrouteservice.Client(key=api_key)

    print(start_coords, end_coords)

    route = client.directions(
        coordinates=[start_coords, end_coords],
        profile='driving-car',
        format='geojson',
        radiuses=[1000, 1000],
        
    )

    # Distancia en metros y duración en segundos
    distance_m = route['features'][0]['properties']['segments'][0]['distance']
    duration_s = route['features'][0]['properties']['segments'][0]['duration']

    # Convertimos a km y minutos
    distance_km = distance_m / 1000
    duration_min = (duration_s / 60) 
    #print(f"Distancia de conducción: {distance_km:.2f} km")
    #print(f"Duración estimada: {duration_hr:.2f} minutos")

    return  distance_km, duration_min

api_key = '5b3ce3597851110001cf62487e04df114afa4c3ab39b2f9422fc450a'

def haversine_distance(coord1, coord2):
    """
    Calculate the haversine distance between two coordinates.
    :param coord1: Tuple of (latitude, longitude) for the first coordinate.
    :param coord2: Tuple of (latitude, longitude) for the second coordinate.
    :return: Distance in kilometers.
    """
    return haversine(coord1, coord2)


In [ ]:
depots = depots.rename(columns={"LocationID": "id"})
clients = clients.rename(columns={"LocationID": "id"})
depots["id"] = 0 
depots["Demand"] = 0

clients = clients.reset_index(drop=True)
clients["id"] = clients.index + 1  

depots = depots[["id", "Longitude", "Latitude", "Demand"]]
clients = clients[["id", "Longitude", "Latitude", "Demand","TimeWindow"]]

nodes = pd.concat([depots, clients], ignore_index=True)


N = nodes["id"].tolist()
V = vehicles["VehicleID"].tolist()
De = nodes.set_index("id")["Demand"].to_dict()
VT = [convertir_hora_a_minutos(i) for i in clients["TimeWindow"].tolist()]
C = clients["id"].tolist()


lat_lon_list = [tuple(coord) for coord in nodes[["Latitude", "Longitude"]].values.tolist()] # lista de tuplas con latitudes y longitudes ordenadas por id
lon_lat_list = [(lon, lat) for lat, lon in lat_lon_list] # lista de tuplas con longitudes y latitudes ordenadas por id

di_haversine = [
    [10000 if i == j else haversine_distance(lat_lon_list[i], lat_lon_list[j]) for i in range(len(lat_lon_list))]
    for j in range(len(lat_lon_list))
] 

# la otra distancia se calcula con offset 30% más que el haversine

di_street = [i*1.3 for i in di_haversine]



# Asumimos velocidad dron de 100 km/h
T_haversine = [
    [10000 if i == j else (haversine_distance(lat_lon_list[i], lat_lon_list[j]) / 100) for i in range(len(lat_lon_list))]
    for j in range(len(lat_lon_list))
]

# Asumimos velocidad de 50km/h para el carro

T_street = [
    [10000 if i == j else (di_street[i][j] / 50) for i in range(len(lat_lon_list))]
    for j in range(len(lat_lon_list))
]


T = [T_street, T_haversine] # matriz de tiempos de viaje entre nodos, primero carro y luego dron
di = [di_street, di_haversine] # matriz de distancias entre nodos, primero carro y luego dron

costos = [0.57, 0.233] # costo por km carro y dron
tipo = [0 if i == "4x4" else 1 for i in vehicles["Type"].tolist()]

depot = 0

P = vehicles.set_index("VehicleID")["Capacity"].to_dict()
M = vehicles.set_index("VehicleID")["Range"].to_dict()






print("Nodos:", N)
print("Clientes:", C)
print("Vehículos:", V)
print("Capacidades:", P)
print("Alcances:", M)
print("Demanda:", De)
print("Rango de tiempo:", VT)
print("latlon", lat_lon_list)
print("Distancias haversine:", di_haversine)
print("Tiempos haversine:", T_haversine)
print("Costos:", costos)
print("Tipo de vehículo:", tipo)